In [13]:
import os
import time
import numpy as np
import torch
import onnx
import onnxruntime as ort
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import pandas as pd
from model_def import *

In [15]:
class CheXpertDataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, image_size=224):
        self.df = pd.read_csv(csv_path)
        self.image_paths = self.df["corrected_path"].values

        # Extract label columns from start to end
        start_col = "Enlarged Cardiomediastinum"
        end_col = "No Finding"
        label_columns = self.df.loc[:, start_col:end_col].columns

        # Load and convert -1 to 1
        self.labels = self.df[label_columns].astype(np.float32).values
        self.labels[self.labels == -1.0] = 1.0  # Convert -1s to 1s

        self.transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.Grayscale(num_output_channels=3),
            transforms.ToTensor(),
            transforms.Normalize([0.5]*3, [0.5]*3)
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # path = self.image_paths[idx].replace("/data/", "/")
        path = self.image_paths[idx].replace("/mnt/data/", "/mnt/dataset/")
        image = Image.open(path).convert("RGB")
        image = self.transform(image)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return image, label

In [16]:
from torch.utils.data import DataLoader, Subset, random_split

csv_path = r"filtered_chexpert_paths.csv"

full_dataset = CheXpertDataset(csv_path)
total_len = len(full_dataset)

# Indices
sixty_percent = int(0.6 * total_len)
next_percent = int(0.62 * total_len)

# First 60% for training/validation
# dataset_60 = Subset(full_dataset, list(range(0, sixty_percent)))

# Middle 30% for testing
dataset_test = Subset(full_dataset, list(range(sixty_percent, next_percent)))

test_loader = DataLoader(dataset_test, batch_size=16, shuffle=False, num_workers=4)

In [8]:
model_path = "./mlflowModel1.pt"  
device = torch.device("cpu")
model = torch.load(model_path, map_location=device, weights_only=False)


onnx_model_path = "./mlflowModel1.onnx"
# dummy input - used to clarify the input shape
dummy_input = torch.randn(1, 3, 224, 224)  
torch.onnx.export(model, dummy_input, onnx_model_path,
                  export_params=True, opset_version=20,
                  do_constant_folding=True, input_names=['input'],
                  output_names=['output'], dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}})

print(f"ONNX model saved to {onnx_model_path}")

onnx_model = onnx.load(onnx_model_path)
onnx.checker.check_model(onnx_model)

ONNX model saved to ./mlflowModel1.onnx


In [17]:
onnx_model_path = "./mlflowModel1.onnx"

ort_session = ort.InferenceSession(onnx_model_path, providers=['CPUExecutionProvider'])

ort_session.get_providers()

['CPUExecutionProvider']

In [18]:
model_size = os.path.getsize(onnx_model_path)
print(f"Model Size on Disk: {model_size / 1e6 :.2f} MB")

Model Size on Disk: 28.28 MB


In [23]:
def benchmark_session(ort_session):
    model_size = os.path.getsize(onnx_model_path)
    print(f"Model Size on Disk: {model_size / 1e6 :.2f} MB")
    
    print(f"Execution provider: {ort_session.get_providers()}")
    correct = 0
    total = 0
    
    for images, labels in test_loader:
        images_np = images.numpy()
        # Run ONNX model inference
        outputs = ort_session.run(None, {ort_session.get_inputs()[0].name: images_np})[0]  # shape: [B, 14]
        # Predicted class: index of max logit
        # predicted = np.argmax(outputs, axis=1)
        preds = (1 / (1 + np.exp(-outputs))) > 0.5
        # If labels are one-hot or multi-hot: use argmax
        # target = np.argmax(labels.numpy(), axis=1)
        # correct += np.sum(predicted == target)
        # total += labels.size(0)
        labels_np = labels.numpy().astype(bool)
        correct += np.sum(preds == labels_np)
        total += labels_np.size
    
    accuracy = (correct / total) * 100
    print(f"ONNX Model Accuracy: {accuracy:.2f}%")

    num_trials = 100  # Number of trials

    # Get a single sample from the test data
    
    single_sample, _ = next(iter(test_loader))  
    single_sample = single_sample[:1].numpy()
    
    # Warm-up run
    ort_session.run(None, {ort_session.get_inputs()[0].name: single_sample})
    
    latencies = []
    for _ in range(num_trials):
        start_time = time.time()
        ort_session.run(None, {ort_session.get_inputs()[0].name: single_sample})
        latencies.append(time.time() - start_time)
    print(f"Inference Latency (single sample, median): {np.percentile(latencies, 50) * 1000:.2f} ms")
    print(f"Inference Latency (single sample, 95th percentile): {np.percentile(latencies, 95) * 1000:.2f} ms")
    print(f"Inference Latency (single sample, 99th percentile): {np.percentile(latencies, 99) * 1000:.2f} ms")
    print(f"Inference Throughput (single sample): {num_trials/np.sum(latencies):.2f} FPS")

    
    num_batches = 50  # Number of trials
    # Get a batch from the test data
    batch_input, _ = next(iter(test_loader))  
    batch_input = batch_input.numpy()
    
    # Warm-up run
    ort_session.run(None, {ort_session.get_inputs()[0].name: batch_input})
    
    batch_times = []
    for _ in range(num_batches):
        start_time = time.time()
        ort_session.run(None, {ort_session.get_inputs()[0].name: batch_input})
        batch_times.append(time.time() - start_time)
        
    batch_fps = (batch_input.shape[0] * num_batches) / np.sum(batch_times) 
    print(f"Batch Throughput: {batch_fps:.2f} FPS")

In [58]:
onnx_model_path = "./mlflowModel1.onnx"
ort_session = ort.InferenceSession(onnx_model_path, providers=['CPUExecutionProvider'])
benchmark_session(ort_session)

Model Size on Disk: 28.28 MB
Execution provider: ['CPUExecutionProvider']
ONNX Model Accuracy: 77.96%
Inference Latency (single sample, median): 25.73 ms
Inference Latency (single sample, 95th percentile): 27.24 ms
Inference Latency (single sample, 99th percentile): 32.52 ms
Inference Throughput (single sample): 38.22 FPS
Batch Throughput: 55.19 FPS


In [29]:
onnx_model_path = "./mlflowModel1.onnx"
optimized_model_path = "./mlflowModel1_optimized.onnx"

session_options = ort.SessionOptions()
session_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_EXTENDED # apply graph optimizations
session_options.optimized_model_filepath = optimized_model_path

ort_session = ort.InferenceSession(onnx_model_path, sess_options=session_options, providers=['CPUExecutionProvider'])

In [30]:
onnx_model_path = "./mlflowModel1_optimized.onnx"
ort_session = ort.InferenceSession(onnx_model_path, providers=['CPUExecutionProvider'])
benchmark_session(ort_session)

Model Size on Disk: 28.27 MB
Execution provider: ['CPUExecutionProvider']
ONNX Model Accuracy: 77.96%
Inference Latency (single sample, median): 25.71 ms
Inference Latency (single sample, 95th percentile): 31.86 ms
Inference Latency (single sample, 99th percentile): 33.59 ms
Inference Throughput (single sample): 36.66 FPS
Batch Throughput: 59.96 FPS


In [31]:
import neural_compressor
from neural_compressor import quantization

# Load ONNX model into Intel Neural Compressor
model_path = "./mlflowModel1.onnx"
fp32_model = neural_compressor.model.onnx_model.ONNXModel(model_path)

# Configure the quantizer
config_ptq = neural_compressor.PostTrainingQuantConfig(
    approach="dynamic"
)

# Fit the quantized model
q_model = quantization.fit(
    model=fp32_model, 
    conf=config_ptq
)

# Save quantized model
q_model.save_model_to_file("./mlflowModel1_quantized_dynamic.onnx")

2025-05-10 20:21:54 [INFO] Start auto tuning.
2025-05-10 20:21:54 [INFO] Quantize model without tuning!
2025-05-10 20:21:54 [INFO] Quantize the model with default configuration without evaluating the model.                To perform the tuning process, please either provide an eval_func or provide an                    eval_dataloader an eval_metric.
2025-05-10 20:21:54 [INFO] Adaptor has 5 recipes.
2025-05-10 20:21:54 [INFO] 0 recipes specified by user.
2025-05-10 20:21:54 [INFO] 3 recipes require future tuning.
2025-05-10 20:21:54 [INFO] *** Initialize auto tuning
2025-05-10 20:21:54 [INFO] {
2025-05-10 20:21:54 [INFO]     'PostTrainingQuantConfig': {
2025-05-10 20:21:54 [INFO]         'AccuracyCriterion': {
2025-05-10 20:21:54 [INFO]             'criterion': 'relative',
2025-05-10 20:21:54 [INFO]             'higher_is_better': True,
2025-05-10 20:21:54 [INFO]             'tolerable_loss': 0.01,
2025-05-10 20:21:54 [INFO]             'absolute': None,
2025-05-10 20:21:54 [INFO]     

In [32]:
onnx_model_path = "./mlflowModel1_quantized_dynamic.onnx"
model_size = os.path.getsize(onnx_model_path) 
print(f"Model Size on Disk: {model_size/ (1e6) :.2f} MB")

Model Size on Disk: 7.86 MB


In [33]:
onnx_model_path = "./mlflowModel1_quantized_dynamic.onnx"
ort_session = ort.InferenceSession(onnx_model_path, providers=['CPUExecutionProvider'])
benchmark_session(ort_session)

Model Size on Disk: 7.86 MB
Execution provider: ['CPUExecutionProvider']
ONNX Model Accuracy: 76.64%
Inference Latency (single sample, median): 45.91 ms
Inference Latency (single sample, 95th percentile): 48.47 ms
Inference Latency (single sample, 99th percentile): 51.35 ms
Inference Throughput (single sample): 21.60 FPS
Batch Throughput: 24.92 FPS


In [49]:

import neural_compressor
from neural_compressor import quantization
from torchvision import datasets, transforms

image_size = 224
test_transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.Grayscale(num_output_channels=3),
            transforms.ToTensor(),
            transforms.Normalize([0.5]*3, [0.5]*3)
        ])

# Load dataset
dataset_test = Subset(full_dataset, list(range(sixty_percent, next_percent)))
test_loader = DataLoader(dataset_test, batch_size=16, shuffle=False, num_workers=4)
eval_dataloader = neural_compressor.data.DataLoader(framework='onnxruntime', dataset=dataset_test)

In [53]:
import onnxruntime as ort
from neural_compressor.model.onnx_model import ONNXModel

onnx_model_path = "./mlflowModel1.onnx"
fp32_session = ort.InferenceSession(onnx_model_path, providers=['CPUExecutionProvider'])
fp32_model = ONNXModel(onnx_model_path)  # still needed for INC's internal use

# Configure the quantizer
config_ptq = neural_compressor.PostTrainingQuantConfig(
    accuracy_criterion = neural_compressor.config.AccuracyCriterion(
        criterion="absolute",  
        tolerable_loss=0.1  # We will tolerate up to 0.01 less accuracy in the quantized model
    ),
    approach="static", 
    device='cpu', 
    quant_level=0,  # 0 is a less aggressive quantization level
    quant_format="QOperator", 
    recipes={"graph_optimization_level": "ENABLE_EXTENDED"}, 
    calibration_sampling_size=128
)

def eval_func(_):
    correct = 0
    total = 0
    threshold = 0.5

    for images, labels in test_loader:
        images_np = images.numpy()
        labels_np = labels.numpy().astype(bool)

        outputs = fp32_session.run(None, {fp32_session.get_inputs()[0].name: images_np})[0]
        preds = (1 / (1 + np.exp(-outputs))) > threshold

        correct += np.sum(preds == labels_np)
        total += labels_np.size

    return correct / total

# Find the best quantized model meeting the accuracy criterion
q_model = quantization.fit(
    model=fp32_model, 
    conf=config_ptq, 
    calib_dataloader=eval_dataloader,
    # eval_dataloader=eval_dataloader, 
    eval_func=eval_func
)

# Save quantized model
q_model.save_model_to_file("./mlflowModel1_quantized_conservative.onnx")

2025-05-10 20:38:43 [INFO] Start conservative tuning.
2025-05-10 20:38:43 [INFO] Execute the tuning process due to detect the evaluation function.
2025-05-10 20:38:43 [INFO] Adaptor has 5 recipes.
2025-05-10 20:38:43 [INFO] 0 recipes specified by user.
2025-05-10 20:38:43 [INFO] 3 recipes require future tuning.
2025-05-10 20:38:43 [INFO] *** Initialize conservative tuning
2025-05-10 20:38:43 [INFO] {
2025-05-10 20:38:43 [INFO]     'PostTrainingQuantConfig': {
2025-05-10 20:38:43 [INFO]         'AccuracyCriterion': {
2025-05-10 20:38:43 [INFO]             'criterion': 'absolute',
2025-05-10 20:38:43 [INFO]             'higher_is_better': True,
2025-05-10 20:38:43 [INFO]             'tolerable_loss': 0.1,
2025-05-10 20:38:43 [INFO]             'absolute': 0.1,
2025-05-10 20:38:43 [INFO]             'keys': <bound method AccuracyCriterion.keys of <neural_compressor.config.AccuracyCriterion object at 0x745f7cb1ffb0>>,
2025-05-10 20:38:43 [INFO]             'relative': None
2025-05-10 20:38

In [54]:
onnx_model_path = "./mlflowModel1_quantized_conservative.onnx"
ort_session = ort.InferenceSession(onnx_model_path, providers=['CPUExecutionProvider'])
benchmark_session(ort_session)

Model Size on Disk: 19.88 MB
Execution provider: ['CPUExecutionProvider']
ONNX Model Accuracy: 78.29%
Inference Latency (single sample, median): 35.38 ms
Inference Latency (single sample, 95th percentile): 39.80 ms
Inference Latency (single sample, 99th percentile): 48.78 ms
Inference Throughput (single sample): 27.75 FPS
Batch Throughput: 35.54 FPS


In [55]:
onnx_model_path = "./mlflowModel1.onnx"
ort_session = ort.InferenceSession(onnx_model_path, providers=['CPUExecutionProvider'])
benchmark_session(ort_session)     

Model Size on Disk: 28.28 MB
Execution provider: ['CPUExecutionProvider']
ONNX Model Accuracy: 77.96%
Inference Latency (single sample, median): 27.09 ms
Inference Latency (single sample, 95th percentile): 29.74 ms
Inference Latency (single sample, 99th percentile): 29.90 ms
Inference Throughput (single sample): 36.42 FPS
Batch Throughput: 60.15 FPS


In [56]:
onnx_model_path = "./mlflowModel1.onnx"
ort_session = ort.InferenceSession(onnx_model_path, providers=['CUDAExecutionProvider'])
benchmark_session(ort_session)
ort.get_device()  

Model Size on Disk: 28.28 MB
Execution provider: ['CUDAExecutionProvider', 'CPUExecutionProvider']


2025-05-10 20:41:18.488273433 [W:onnxruntime:, transformer_memcpy.cc:83 ApplyImpl] 6 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2025-05-10 20:41:18.493223682 [W:onnxruntime:, session_state.cc:1280 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-05-10 20:41:18.493236297 [W:onnxruntime:, session_state.cc:1282 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


ONNX Model Accuracy: 77.96%
Inference Latency (single sample, median): 10.15 ms
Inference Latency (single sample, 95th percentile): 12.54 ms
Inference Latency (single sample, 99th percentile): 14.51 ms
Inference Throughput (single sample): 95.33 FPS
Batch Throughput: 376.29 FPS


'GPU'

In [ ]:
#P100 GPU, TesnorRT does not work
onnx_model_path = "./mlflowModel1.onnx"
ort_session = ort.InferenceSession(onnx_model_path, providers=['TensorrtExecutionProvider'])
benchmark_session(ort_session)
ort.get_device()  

Model Size on Disk: 28.28 MB
Execution provider: ['CPUExecutionProvider']


2025-05-10 20:46:59.718799559 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:880 CreateExecutionProviderInstance] Failed to create TensorrtExecutionProvider. Please reference https://onnxruntime.ai/docs/execution-providers/TensorRT-ExecutionProvider.html#requirements to ensure all dependencies are met.
